## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob

## Read-in

In [2]:
# Get today's date
today = datetime.now()

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Jun. 4th Jun. 11th


In [19]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [20]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [21]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [23]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [28]:
# Filter dates between June 3 and June 11, inclusive
df = df[(df['SOLD DATE'] >= '2024-06-03') & (df['SOLD DATE'] <= '2024-06-11')]

In [29]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
166,PAST SALE,2024-06-11,Condo/Co-op,2889 Mcfarlane Rd #2017,Coconut Grove,FL,33133.0,915000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2889-McFarlane...,MARMLS,A11524797,N,Y,25.727274,-80.240539
217,PAST SALE,2024-06-10,Condo/Co-op,133 NE 2nd Ave #2409,Miami,FL,33132.0,365000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/133-NE-2nd-Ave...,MARMLS,A11532561,N,Y,25.775954,-80.190193
157,PAST SALE,2024-06-10,Condo/Co-op,2843 S Bayshore Dr Unit 17A,Coconut Grove,FL,33133.0,1000000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2843-S-Bayshor...,MARMLS,A11513177,N,Y,25.728269,-80.238889
89,PAST SALE,2024-06-10,Condo/Co-op,10 Venetian Way Ph 1,Miami Beach,FL,33139.0,6000000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/10-Venet...,MARMLS,A11476722,N,Y,25.791132,-80.146640
92,PAST SALE,2024-06-10,Condo/Co-op,5959 Collins Ave #1107,Miami Beach,FL,33140.0,4855000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5959-Col...,MARMLS,A11486809,N,Y,25.841880,-80.120969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,PAST SALE,2024-06-03,Condo/Co-op,6930 Miami Gardens Dr Unit 1-203,Hialeah,FL,33015.0,220000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hialeah/6930-Miami-G...,MARMLS,A11551153,N,Y,25.941488,-80.315485
100,PAST SALE,2024-06-03,Condo/Co-op,16400 Collins Ave #1046,Sunny Isles Beach,FL,33160.0,1390000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,Beaches MLS,RX-10903966,N,Y,25.928467,-80.123797
187,PAST SALE,2024-06-03,Condo/Co-op,1990 Marseille Dr #406,Miami Beach,FL,33141.0,330000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1990-Mar...,MARMLS,A11517676,N,Y,25.852731,-80.141965
164,PAST SALE,2024-06-03,Condo/Co-op,16850 SW Glades Dr Unit 4J,North Miami Beach,FL,33162.0,100000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami-Beach/16...,Beaches MLS,F10421495,N,Y,25.930760,-80.169415


In [30]:
df = df.sort_values(by='PRICE',ascending=False)
df_top_ten = df.head(10)

## Current Week's Values

In [31]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

189


In [32]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$892,258


In [33]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$575


In [34]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$168,636,815


In [35]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [54]:
# # Define the date range
# start_date = '2024-05-20'
# end_date = '2024-05-26'

# # Filter rows where 'SOLD DATE' is within the specified range
# second_week_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

## Previous Week's Values

In [37]:
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

Input Previous Week Condo Sales Total (number units sold):
152
Input Previous Week Condo Average Sales Price:
830000
Input Previous Week Condo Average PSF:
580
Input Previous Week Condo Sales total (ex: 198_000_000)
126_000_000


## Clean Data

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [38]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [39]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [40]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'RANK', 'COLOR'],
      dtype='object')

In [41]:
pd.set_option('display.max_columns',None)

In [42]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR
82,PAST SALE,2024-06-03,Condo/Co-op,4701 N Meridian Ave Ph 01,Miami Beach,FL,33140.0,17250000.0,4.0,4.5,Ritz Carlton Residences,5936.0,NaN,2019.0,NaN,2906.0,12753.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4701-Mer...,Beaches MLS,F10413035,N,Y,25.822762,-80.132162,1,orange


In [43]:
def popup_html(row):
    building_name = row['LOCATION']
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [44]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [45]:
m.save('index.html')

## Data snagger

In [46]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [47]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR
82,PAST SALE,2024-06-03,Condo/Co-op,4701 N Meridian Ave Ph 01,Miami Beach,FL,33140.0,17250000.0,4.0,4.5,Ritz Carlton Residences,5936.0,NaN,2019.0,NaN,2906.0,12753.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4701-Mer...,Beaches MLS,F10413035,N,Y,25.822762,-80.132162,1,orange


In [ ]:
# , {df_top_ten[df_top_ten['PRICE'].idxmax()]['ADDRESS']} | Price ${df_top_ten[df_top_ten['PRICE'].idxmax()]['PRICE']:,.0f}

In [48]:
df_top_ten['PRICE'].iloc[0]

17250000.0

In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [49]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_06112024


# CREATE TEMPLATE 

In [50]:
muni_set = set(df_top_ten['CITY'])

In [51]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [52]:
df_top_ten.reset_index(inplace=True,drop=True)

In [53]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [54]:
top_sale

'Ritz Carlton Residences at 4701 N Meridian Ave Ph 01 in Miami Beach'

In [29]:
# previous_week_condo_sales_total = 130_000_000

In [55]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [111]:
# df_top_ten

## RESUME HERE

In [56]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops Miami-Dade weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m Miami-Dade County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m Miami-Dade County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #MiamiDade condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, Miami-Dade County, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

Miami-Dade County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} sales average from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Ritz Carlton Residences closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,900,000M to $17,250,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Bal Harbour, Sunny Isles Beach, Miami Beach, Key Biscayne

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 189 condo sales totaling $168,636,815 million from Jun. 4th to Jun. 11th. The previous week, brokers closed 152 condo sales totaling $126,000,000.

Last week’s units sold for an average of $892,258, higher than the $830,000 sales average from the previous week. The 

In [57]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [58]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [59]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf")

print(story_checklist)


HED: Ritz Carlton Residences closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $2,900,000M to $17,250,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Bal Harbour, Sunny Isles Beach, Miami Beach, Key Biscayne

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 189 condo sales totaling $168,636,815 million from Jun. 4th to Jun. 11th. The previous week, brokers closed 152 condo sales totaling $126,000,000.

Last week’s units sold for an average of $892,258, higher than the $830,000 sales average from the previous week. The 

In [67]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Miami-Beach/5775-Collins-Ave-33140/unit-1203/home/174243437


In [69]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Miami-Beach/300-Collins-Ave-33139/unit-4D/home/148119396


In [62]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Sunny-Isles-Beach/16901-Collins-Ave-33160/unit-3903/home/56743930


In [63]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Miami-Beach/5775-Collins-Ave-33140/unit-1203/home/174243437


In [70]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Miami-Beach/10-Venetian-Way-33139/unit-1/home/178261534


In [65]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-06-03,Condo/Co-op,4701 N Meridian Ave Ph 01,Miami Beach,FL,33140.0,17250000.0,4.0,4.5,Ritz Carlton Residences,5936.0,NaN,2019.0,NaN,2906.0,12753.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4701-Mer...,Beaches MLS,F10413035,N,Y,25.822762,-80.132162,1,orange,Ritz Carlton Residences at 4701 N Meridian Ave...
1,PAST SALE,2024-06-06,Condo/Co-op,5775 Collins Ave #1203,Miami Beach,FL,33140.0,12000000.0,4.0,5.5,AMD PL OF 2ND OCEAN FRONT,3346.0,NaN,2021.0,NaN,3586.0,9195.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5775-Col...,MARMLS,A11483405,N,Y,25.839257,-80.120395,2,blue,AMD PL OF 2ND OCEAN FRONT at 5775 Collins Ave ...
2,PAST SALE,2024-06-06,Condo/Co-op,10203 Collins Ave #1402,Bal Harbour,FL,33154.0,9500000.0,3.0,4.5,OCEANA BAL HARBOUR CONDO,3450.0,NaN,2016.0,NaN,2754.0,7277.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10203-Co...,MARMLS,A11529070,N,Y,25.895557,-80.122903,3,blue,OCEANA BAL HARBOUR CONDO at 10203 Collins Ave ...
3,PAST SALE,2024-06-10,Condo/Co-op,10201 Collins Ave #802,Bal Harbour,FL,33154.0,6300000.0,2.0,3.0,OCEANA BAL HARBOUR CONDO,2121.0,NaN,2016.0,NaN,2970.0,3610.0,Sold,NaN,NaN,https://www.redfin.com/FL/Bal-Harbour/10201-Co...,MARMLS,A11514106,N,Y,25.895168,-80.123150,4,blue,OCEANA BAL HARBOUR CONDO at 10201 Collins Ave ...
4,PAST SALE,2024-06-10,Condo/Co-op,10 Venetian Way Ph 1,Miami Beach,FL,33139.0,6000000.0,3.0,2.5,THE GRAND VENETIAN CONDO,4750.0,NaN,2001.0,NaN,1263.0,5320.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/10-Venet...,MARMLS,A11476722,N,Y,25.791132,-80.146640,5,blue,THE GRAND VENETIAN CONDO at 10 Venetian Way Ph...
5,PAST SALE,2024-06-05,Condo/Co-op,1000 S Pointe Dr Unit LPH04,Miami Beach,FL,33139.0,5750000.0,3.0,3.0,MURANO AT PORTOFINO CONDO,2190.0,NaN,2002.0,NaN,2626.0,3024.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/1000-S-P...,MARMLS,A11555045,N,Y,25.768527,-80.137591,6,blue,MURANO AT PORTOFINO CONDO at 1000 S Pointe Dr ...
6,PAST SALE,2024-06-10,Condo/Co-op,5959 Collins Ave #1107,Miami Beach,FL,33140.0,4855000.0,3.0,3.5,THE RESIDENCES AT THE BAT,2959.0,NaN,2005.0,NaN,1641.0,4558.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/5959-Col...,MARMLS,A11486809,N,Y,25.841880,-80.120969,7,blue,THE RESIDENCES AT THE BAT at 5959 Collins Ave ...
7,PAST SALE,2024-06-10,Condo/Co-op,791 Crandon Blvd #407,Key Biscayne,FL,33149.0,4710000.0,4.0,5.5,OCEAN TOWER TWO CONDO,3581.0,NaN,2001.0,NaN,1315.0,4786.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/791-Cra...,MARMLS,A11569824,N,Y,25.688337,-80.157932,8,blue,OCEAN TOWER TWO CONDO at 791 Crandon Blvd #407...
8,PAST SALE,2024-06-04,Condo/Co-op,16901 Collins Ave #3903,Sunny Isles Beach,FL,33160.0,4000000.0,4.0,4.5,JADE SIGNATURE CONDO,2643.0,NaN,2018.0,NaN,1513.0,4624.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11519818,N,Y,25.932764,-80.121036,9,blue,JADE SIGNATURE CONDO at 16901 Collins Ave #390...
9,PAST SALE,2024-06-07,Condo/Co-op,300 Collins Ave Unit 4D,Miami Beach,FL,33139.0,2900000.0,3.0,3.5,300 COLLINS CONDO,1699.0,NaN,2018.0,NaN,1707.0,4492.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/300-Coll...,MARMLS,A11507200,N,Y,25.772642,-80.133687,10,blue,300 COLLINS CONDO at 300 Collins Ave Unit 4D i...


## Time on Market Calculator

In [68]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2023, 11, 9) ## List (Earlier) date
date2 = datetime(2024, 6, 6) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

210
